In [15]:
import pathlib
import datetime
import calendar
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
from sklearn.base import TransformerMixin

data_folder = pathlib.Path('data')

## prepare data

In [3]:
train = pd.read_csv(data_folder.joinpath('data_train.csv'), index_col='Unnamed: 0')
train.head(2)

,id,vas_id,buy_time,target
0,540968,8.0,1537131600,0.0
1,1454121,4.0,1531688400,0.0


In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 831653 entries, 0 to 831652
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        831653 non-null  int64  
 1   vas_id    831653 non-null  float64
 2   buy_time  831653 non-null  int64  
 3   target    831653 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 31.7 MB


In [4]:
class TimestampRecast:
    """ Recast timestamp to datetime """
    def fit_transform(self, X, y=None, **fit_params):
        parse_buy_time = np.vectorize(datetime.datetime.fromtimestamp)
        df = X.copy()
        df['buy_time'] = parse_buy_time(df['buy_time'])
        return df

In [5]:
class SimplestPreparer(TransformerMixin):
    """ The simplest data transformations """

    def fit_transform(self, X, y=None, **fit_params):
        parse_buy_time = np.vectorize(datetime.datetime.fromtimestamp)
        df = X.copy()
        # recast types
        df['vas_id'] = df['vas_id'].astype('int')
        if 'target' in df.columns:
            df['target'] = df['target'].astype('int')

        # parse buy timestamp
        df['buy_time'] = parse_buy_time(df['buy_time'])
        # index weeks
        # df['week_no'] = df['buy_time'].astype('category').cat.codes + 1
        # get months
        df['buy_time'].apply(lambda val: val.month)
        return df

In [6]:
train = pd.read_csv(data_folder.joinpath('data_train.csv'), index_col='Unnamed: 0')
train = SimplestPreparer().fit_transform(train)
train.head(2)

,id,vas_id,buy_time,target
0,540968,8,2018-09-17,0
1,1454121,4,2018-07-16,0


In [62]:
test = pd.read_csv(data_folder.joinpath('data_test.csv'), index_col='Unnamed: 0')
test = SimplestPreparer().fit_transform(test)
test.head(2)

,id,vas_id,buy_time
0,3130519,2,2019-01-21
1,2000860,4,2019-01-21


In [7]:
# # save
# train['buy_time'] = train['buy_time'].apply(lambda val: str(val.date()))
# train.to_csv(data_folder.joinpath('train_prepared.csv'), index=False)

## Exploratory data analysis

Ниже представлена только программная часть анализа. Графики представлены в дашбордах:
https://public.tableau.com/app/profile/peter3691/viz/megafon

In [8]:
# train users count
print(f"Users count in train: {train['id'].nunique()}")
print(f"Users count in test: {test['id'].nunique()}")

# cold users
cold_users_mask = test['id'].isin(train['id'].unique())
print(f"Cold users count in test: {test.loc[cold_users_mask, 'id'].nunique()}")

# vas_id difference
diff = set(train['vas_id']) ^ set(test['vas_id'])
print(f"Products train/test difference: {diff if diff else None}")

# total products count
print(f"Total products count: {train['vas_id'].nunique()}")

Users count in train: 806613
Users count in test: 70152
Cold users count in test: 4188
Products train/test difference: None
Total products count: 8


In [9]:
# count weekdays
train['buy_time'].apply(lambda val: calendar.day_name[val.weekday()]).value_counts()

Monday    831653
Name: buy_time, dtype: int64

In [10]:
# min/max date
mn, mx = train['buy_time'].min(), train['buy_time'].max()
print(f'First date: {mn.date()}', f'Last date:  {mx.date()}', sep='\n')
print(f"Months in data: {train['buy_time'].apply(lambda val: val.month).nunique()}")
print(f"Weeks in data: {train['buy_time'].nunique()}")

First date: 2018-07-09
Last date:  2018-12-31
Months in data: 6
Weeks in data: 26


в buy_time только понедельники. Видимо по понедельникам отрабатывает пайплайн аггрегации данных.
Следовательно, изучить данные в разрезе дней недели не выйдет.
Данные представлены только за часть 2018 года и охватывают разброс в 6 месяцев.
Поскольку данные уже аггрегированы по неделям, динамика за месяц может быть смазана, поэтому буду рассматривать динамику по неделям. 
Для удобства проиндексирую недели, начиная с единицы.

In [11]:
# date of first/last sale
first = train.groupby('vas_id')['buy_time'].min()
last = train.groupby('vas_id')['buy_time'].max()
pd.DataFrame([first, last], index=['first', 'last']).T

,first,last
vas_id,,
1,2018-07-09,2018-12-31
2,2018-07-09,2018-12-31
4,2018-07-09,2018-12-31
5,2018-07-09,2018-12-31
6,2018-07-09,2018-12-31
7,2018-08-27,2018-12-31
8,2018-07-09,2018-12-31
9,2018-07-09,2018-12-31


Даты первой и последней продаж совпадают с границами датасета практически для всех услуг

In [84]:
# aggregate offers data
offers = train.sort_values(by='buy_time').groupby(['vas_id', 'id'])['target'].agg(list)

# Max number of offers
print(f'Max number of offers to one user: {offers.apply(len).max()}')

# Double offers analyse
double_offer_mask = offers.apply(len) > 1

# Users whose target has remained
target_remain_mask = offers[double_offer_mask].apply(lambda val: val[0] == val[1])

print(f'Number of users who left their target unchanged: {offers[double_offer_mask][target_remain_mask].size}')
remained = offers[double_offer_mask][target_remain_mask]
pos_mask = remained.apply(lambda val: val[1] == 1)
print(f'    remained 0: {remained[~pos_mask].size}', f'    remained 1: {remained[pos_mask].size}', sep='\n')

# Users whose target has changed
print(f'Number of users who changed their target: {offers[double_offer_mask][~target_remain_mask].size}')
changed = offers[double_offer_mask][~target_remain_mask]
pos_mask = changed.apply(lambda val: val[1] == 1)
print(f'    changed to 0: {changed[~pos_mask].size}', f'    changed to 1: {changed[pos_mask].size}', sep='\n')


Max number of offers to one user: 2
Number of users who left their target unchanged: 43
    remained 0: 4
    remained 1: 39
Number of users who changed their target: 6206
    changed to 0: 2160
    changed to 1: 4046


In [14]:
# # draft to keep actual (last) user choice
# dup_mask = train.sort_values(by='buy_time')[['id', 'vas_id']].duplicated(keep='last')
# train.sort_values(by='buy_time')[~dup_mask]

## features

In [19]:
feats = dd.read_csv(data_folder.joinpath('features.csv'), sep='\t').drop('Unnamed: 0', axis=1)
# feats = TimestampRecast().fit_transform(feats)

In [18]:
feats = TimestampRecast().fit_transform(feats)      # TODO

In [ ]:
feats.head()

In [ ]:
# prepare feats headers - key fields of user features
feat_heads = feats[['id', 'buy_time']].compute()
feat_heads.to_csv(data_folder.joinpath('feat_heads.csv'), index=False)

In [ ]:
# read feat headers
# feat_heads = pd.read_csv(data_folder.joinpath('feat_heads.csv'))
feat_heads = TimestampRecast().fit_transform(feat_heads)

In [45]:
# check for duplicated user-time pairs
print(f'Features data contains only unique id-time pairs: {feat_heads.shape[0] == feat_heads.drop_duplicates().shape[0]}')

Features data contains only unique id-time pairs: True


In [48]:
# check weekdays in user features data
feat_heads['buy_time'].apply(lambda val: calendar.day_name[val.weekday()]).unique()

array(['Monday'], dtype=object)

In [44]:
# select users with more than one features' set
# feats_count = feats.groupby(['id'])['buy_time'].nunique()
# print(f'Max count of sets of features for one user: {feats_count.max().compute()}') - очень долго!!!

feats_count = feat_heads.groupby(['id'])['buy_time'].nunique()
print(f'Max count of sets of features for one user: {feats_count.max()}')

Max count of sets of features for one user: 2


In [70]:
# comparing dates in train and features
# dup_feats_users = train.loc[train['id'].isin(feats_count.index[feats_count > 1]), 'id'].unique()
users_with_feats = train.loc[train['id'].isin(feats_count.index), 'id'].unique()
usr_mask = train['id'].isin(users_with_feats)
uft_mask = feat_heads['id'].isin(users_with_feats)

A = train[usr_mask].sort_values(by=['id', 'buy_time'])
B = feat_heads[uft_mask].sort_values(by=['id', 'buy_time'])

# merge and compare buy_time in train and features data
A.merge(B, on='id', how='left', suffixes=('_train', '_feat'))

,id,vas_id,buy_time_train,target,buy_time_feat
0,2,2,2018-12-24,0,2018-12-24
1,4,1,2018-08-06,0,2018-10-01
2,15,1,2018-08-13,0,2018-07-16
3,16,2,2018-10-29,0,2019-01-21
4,29,1,2018-08-06,0,2018-07-30
...,...,...,...,...,...
860047,4362634,1,2018-12-31,0,2018-12-17
860048,4362640,2,2018-12-31,0,2018-12-31
860049,4362647,6,2018-12-31,0,2018-09-24
860050,4362684,5,2018-12-24,0,2018-10-08


Все наборы фичей имеют уникальные пары user-time. Максимальное количество наборов фичей для юзера - `2`. Агрегация, как и в тренировочных данных - по понедельникам. Но дата аггрегации фичей может отличаться от даты покупки в любую сторону.

Если юзерфичи аггрегированы после покупки, то по идее они уже не могли влиять на таргет.

In [ ]:
# TODO ниже сделать так как оно должно работать в модели

In [ ]:
# # select required user features
# used_feats_mask = feats['id'].isin(train['id'].unique())
# user_feats = feats[train_user_feats]

In [ ]:
# merge by nearest aggregation date BEFORE purchase: feats aggregated after purchase date couldn't influence the user's decision 
merged = dd.merge_asof(train.sort_values(by='buy_time'),
                       feats.sort_values(by='buy_time'),
                       by='id', on='buy_time', direction='backward')
# TODO fillna

In [93]:
# # пример работы pd.merge_asof
# merged = pd.merge_asof(A.sort_values(by='buy_time'), B.sort_values(by='buy_time'), by='id', on='buy_time', direction='backward')
# merged.sort_values(by=['id', 'buy_time'])

,id,vas_id,buy_time,target,check,check_date
10425,372,4,2018-09-03,0,NaN,NaT
9407,404,1,2018-08-27,0,-1.0,2018-08-06
17635,487,4,2018-11-05,0,NaN,NaT
12789,620,1,2018-09-24,0,-1.0,2018-09-24
8508,748,5,2018-08-20,0,-1.0,2018-07-16
...,...,...,...,...,...,...
19578,4361764,1,2018-11-19,0,-1.0,2018-07-16
26922,4361965,5,2018-12-31,0,-1.0,2018-11-05
16170,4362012,2,2018-10-22,0,-1.0,2018-08-06
25130,4362200,6,2018-12-24,0,-1.0,2018-11-19


## drafts

## conclusion

Присутствует сильный дисбаланс классов: `7.24%` объектов первого класса

Услуги 6, 4 и 9 продавались лучше всего: `42,68%`, `25,38%` и `18,35%` от количества их предложений было продано. У остальных услуг доля продаж не превышает `2,6%`.
При этом услуга 9 составляет очень малую долю (`1,67%`) в общем количестве продаж.

До 12-19 ноября основные продаваемые услуги были 4, 6 и 9, при этом услуга 4 постепенно вытесняла услугу 6. Потом доля услуги 4 значительно сократилась, но также значительно выросла доля услуги 6, т.е. произошло обратное замещение. Остальные услуги продавались в незначительно малом количестве.

Услуга 9 показала необъяснимый всплеск продаж 19 ноября - на два порядка выше медианы.

Присутствуют повторные предложения как с изменением таргета, так и без. Больше двух раз услуга никому не предлагалась.

В тестовых данных присутствуют записи для `4188` пользователей, которые не были представлены в тренировочных данных (cold users).
Различий по услугам между тренировочными и тестовыми данными нет.

1. Непонятно, с чего такой огромный всплеск продаж услуги 9, достоверен ли он - не понять никак
2. В данных есть повторные офферы с разным таргетом.
    - оставить как есть
    - взять только последний (актуальный)
    - сопоставить с профилем пользователя в указанный период времени (если есть такая возможность)

Задачи
1. Предсказание вероятности подключения услуги
2. Формирование персонального пакета предложений

In [16]:
#